**For comparing two locations**

Based on Folium docs at https://python-visualization.github.io/folium/latest/index.html

In [1]:
import folium
import pandas as pd
from IPython.display import clear_output

#file = input("Enter the excel file name: ")
file = "20rows.xlsx"
#data are specimens sorted with two locations > 50km apart. Maybe reduce this to some other distance?
df = pd.read_excel(file)

nrows = df.shape[0]
row = 1

#print(f"No. specimens is {nrows}")

In [2]:
#needed since the 'spec' variable doesn't seem to remain an integer all the time. Not sure why. 
def is_int(input_str):
    try:
        int(input_str)
        return True
    except ValueError:
        return False

In [3]:
#set specimen number based on entered value
def set_row(inp, row, nrows):
    #print(f"row in set_spec = {row}")
    if is_int(inp):
        #print(f"'{inp}' is an integer.")
        return inp
    else:
        #print(f"'{inp}' is a char.")
        if inp == "g" or inp == "o" or inp == "u" or inp == "d" or inp == "x":
            row += 1
        elif inp == "n": 
            row += 1
        elif inp == "p":
            row -= 1
        elif inp == "f":
            row = 1
        elif inp == "l":
            row = nrows
        else:
            row = -1
    return int(row)

In [4]:
def make_map(row, df):
    #print(f"row is {row}")
    lat1 = df.OrigLat[row-1]
    lon1 = df.OrigLong[row-1]
    lat2 = df.ApLat[row-1]
    lon2 = df.ApLong[row-1]
    trueSpec = df.Specimen[row-1]
    #distance = df.distance[row-1]
    
    if lat1 == 0 and lat2 == 0:
        m = folium.Map(location=[lat2, lon2], zoom_start=5, control_scale=True)
        folium.Marker(
            location=[0, 0],             
            tooltip = "No Location found", 
            icon=folium.Icon(color="red"),
        ).add_to(m)  
    
    elif lat1 == 0 :
        m = folium.Map(location=[lat2, lon2], zoom_start=5, control_scale=True)
        folium.Marker(
            location=[lat2, lon2], 
            tooltip = f"Specimen ID {trueSpec}",
            popup=f"No Original, GeoApp: {lat2}, {lon2}", 
            icon=folium.Icon(color="green"),
        ).add_to(m)        

    elif lat2 == 0 :
        m = folium.Map(location=[lat1, lon1], zoom_start=5, control_scale=True)
        folium.Marker(
            location=[lat1, lon1], 
            tooltip = f"Specimen ID {trueSpec}",
            popup=f"No GeoApp, Original: {lat1}, {lon1}",
            icon=folium.Icon(color="blue"),
        ).add_to(m)
    
    else:        
        m = folium.Map(location=[lat2, lon2], zoom_start=5,control_scale=True)
        #see https://python-visualization.github.io/folium/latest/user_guide/features/fit_overlays.html
        fg1 = folium.FeatureGroup().add_to(m)
        folium.Marker(
            location=[lat1, lon1], 
            tooltip = f"Specimen ID {trueSpec}",
            popup=f"Original: {lat1}, {lon1}",
            icon=folium.Icon(color='blue', icon='1', prefix='fa'),
        ).add_to(fg1)
        
        fg2 = folium.FeatureGroup().add_to(m)
        folium.Marker(
            location=[lat2, lon2], 
            tooltip = f"Specimen ID {trueSpec}",
            popup=f"GeoApp: {lat2}, {lon2}", 
            icon=folium.Icon(color='green', icon='2', prefix='fa'),
        ).add_to(fg2)  

        folium.FitOverlays(max_zoom=7, padding=50).add_to(m)
        folium.LayerControl().add_to(m)
        
    return m

In [5]:
def is_int(n):
    try:
        int(n)
    except ValueError:
        return False
    else:
        return True

In [6]:
#output dataframe to be same as input but with two extra columns to take chosen lat/long
dfout = df

#make the index for both dataframes equal to the specimen ID so that
#     changes can be made by refering to specimen number rather than dataframe row number
#Turns out this isn't needed if update is to specimen that is currently displayed.
#In fact it causes confusion with dataframe indexing. 
#df.index = df["Specimen"]
#dfout.index = dfout["Specimen"]

#add columns for choice of lat/long
dfout['SelectLat'] = ''
dfout['SelectLong'] = ''

#Prove that index is now specimen ID.
#distance = dfout["distance"]
#distance

#show selected values for specimen ID = 472 only
#print(df.loc[472, 'ApLat'], df.loc[472, 'ApLong'])

In [7]:
row = 1
inp = ""

while row >= -1:    
    while row > nrows or row < 1:
        print(f"\n OOPS - CHOICE {row} IS INVALID. Please try again.")
        inp = input(f"Enter row # (1-{nrows}) or (n)ext, (p)rev, (f)irst, (l)ast; \n\
Or choose how to keep loc'n: (o)riginal (marker 1), (g)eoapp (marker 2), (d)ecide later, or (u)nknown (set to 0): ")
        row = int(set_row(inp, int(row), nrows))
        
    clear_output(wait=True)
    m = make_map(row,df)
    display(m)
    #m.save("map.html")
    separation = df.distance[row-1]
    if inp == "x":
            break
    elif inp == "g":
        print("GEOAPPIFY'S LOCATION (2) is kept.\n")
        dfout.loc[row-1, 'SelectLat'] = df.loc[row-1, 'ApLat']
        dfout.loc[row-1, 'SelectLong'] = df.loc[row-1, 'ApLong']
    elif inp == "o":
        print("ORIGINAL LOCATION (1) is kept.\n")
        dfout.loc[row-1, 'SelectLat'] = df.loc[row-1, 'OrigLat']
        dfout.loc[row-1, 'SelectLong'] = df.loc[row-1, 'OrigLong']
    elif inp == "d":
        print("LOCATION flagged is to be decided later.\n")
        dfout.loc[row-1, 'SelectLat'] = "undecided"
        dfout.loc[row-1, 'SelectLong'] = "undecided"
    elif inp == "u":
        print("LOCATION is set to unknown (lat/lon = 0/0).\n")
        dfout.loc[row-1, 'SelectLat'] = 0.0
        dfout.loc[row-1, 'SelectLong'] = 0.0
    
    if is_int(separation):
        print(f"Specimen ID {df.Specimen[row-1]} in row {row}; loc'ns are ~{int(separation)}km apart.") 
    else:
        print(f"Specimen ID {df.Specimen[row-1]} in row {row}. One loc'n only.")
    
    print(f"ADDR={df.address[row-1]}, CITY={df.City[row-1]}, STATE={df.State[row-1]}, COUNTRY={df.Country[row-1]}.")
    
    inp = input(f"ENTER row # (1-{nrows}) or (n)ext, (p)rev, (f)irst, (l)ast;\n\
OR CHOOSE which loc'n to keep: (o)riginal (1), (g)eoapp (2), (d)ecide later, or (u)nknown (set to 0): ")
    row = int(set_row(inp, int(row), nrows))
#end of while loop
    

In [8]:
#save results 
#outf = open("ChosenGeolocns.xlsx", "w", encoding="utf-16") #re-write, NOT appending.
dfout.to_excel("ChosenGeolocns.xlsx")
#outf.close()

clear_output(wait=True)
print("All done.")

All done.
